
# 📊 Projeto de Automação e Análise de Dados de Contratos e Convênios

**Autor:** Artenio Reis  
**Disciplina:** Análise e Automação de Dados  
**Objetivo:** Demonstrar um processo completo de:
- Leitura de dados de contratos e convênios
- Tratamento e padronização das informações
- Criação de banco de dados (SQL Server)
- Análise com métricas e gráficos relevantes
- Exportação de relatório
- Envio automático por e-mail diariamente

---


##  Importação de Bibliotecas

In [64]:

import os, re, unicodedata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.parse import quote_plus




## Coleta de Dados via API

In [65]:
import requests
import pandas as pd

# URLs das APIs
URL_API_contratos = "https://api-dados-abertos.cearatransparente.ce.gov.br/transparencia/contratos/contratos?page=50&data_assinatura_inicio=01%2F01%2F2024&data_assinatura_fim=31%2F05%2F2024"
URL_API_convenios = "https://api-dados-abertos.cearatransparente.ce.gov.br/transparencia/contratos/convenios?page=5&data_assinatura_inicio=01%2F01%2F2024&data_assinatura_fim=31%2F12%2F2024"

# Requisição para Contratos
print("📥 Buscando dados de CONTRATOS...")
response_contratos = requests.get(URL_API_contratos)
if response_contratos.status_code == 200:
    dados_contratos = response_contratos.json()
    contratos_df = pd.DataFrame(dados_contratos.get('data', []))
    contratos_df.to_csv("contratos_df.csv", index=False, encoding='utf-8')
    print("✅ Dados de CONTRATOS obtidos com sucesso!")
    print(f"📂 Arquivo 'contratos_df.csv' salvo com {len(contratos_df)} registros!")
else:
    print(f"❌ Erro ao acessar API de Contratos: {response_contratos.status_code}")

# Requisição para Convênios
print("\n📥 Buscando dados de CONVÊNIOS...")
response_convenios = requests.get(URL_API_convenios)
if response_convenios.status_code == 200:
    dados_convenios = response_convenios.json()
    convenios_df = pd.DataFrame(dados_convenios.get('data', []))
    convenios_df.to_csv("convenios_df.csv", index=False, encoding='utf-8')
    print("✅ Dados de CONVÊNIOS obtidos com sucesso!")
    print(f"📂 Arquivo 'convenios_df.csv' salvo com {len(convenios_df)} registros!")
else:
    print(f"❌ Erro ao acessar API de Convênios: {response_convenios.status_code}")

print("\n🎉 Exportação concluída! Dois arquivos foram gerados:")
print("   - contratos_tratados.csv")
print("   - convenios_df.csv")

📥 Buscando dados de CONTRATOS...
✅ Dados de CONTRATOS obtidos com sucesso!
📂 Arquivo 'contratos_df.csv' salvo com 100 registros!

📥 Buscando dados de CONVÊNIOS...
✅ Dados de CONVÊNIOS obtidos com sucesso!
📂 Arquivo 'convenios_df.csv' salvo com 25 registros!

🎉 Exportação concluída! Dois arquivos foram gerados:
   - contratos_tratados.csv
   - convenios_df.csv


##  Leitura e União das Bases de Dados

In [66]:

# Caminhos dos arquivos CSV exportados
path_contratos = "contratos_df.csv"
path_convenios = "convenios_df.csv"

# Leitura
df_contratos = pd.read_csv(path_contratos, dtype=str, low_memory=False)
df_convenios = pd.read_csv(path_convenios, dtype=str, low_memory=False)

print("Contratos:", df_contratos.shape)
print("Convênios:", df_convenios.shape)

# União em um único DataFrame
df_all = pd.concat([df_contratos, df_convenios], ignore_index=True, sort=False)
df_all.head()


Contratos: (100, 65)
Convênios: (25, 65)


,id,cod_concedente,cod_financiador,cod_gestora,cod_orgao,cod_secretaria,descricao_modalidade,descricao_objeto,descricao_tipo,descricao_url,...,data_inicio,data_rescisao,confidential,gestor_contrato,data_finalizacao_prestacao_contas,sequential,emergency,law,has_non_profit_transfer,nome_fiscal
0,546307,270001,891633,270401,27200004,27000000,INEXIGIBILIDADE,Constitui objeto do presente Contrato a presta...,CONTRATO,20240321.1310038.Integra.CONTRATO.pdf,...,2024-03-20T00:00:00.000-03:00,NaN,False,CAMILA VIEIRA DA SILVA,NaN,5257,False,NaN,NaN,NaN
1,546632,228549,904377,220001,22000000,22000000,DISPENSA,AQUISIÇÃO DE GÊNEROS ALIMENTÍCIOS (DISPENSA ) ...,CONTRATO,20240321.1309998.Integra.CONTRATO.pdf,...,2024-04-02T00:00:00.000-03:00,NaN,False,ROSANGELA PAIXAO VIEIRA DA SILVA,NaN,5531,False,NaN,NaN,NaN
2,547282,240301,914435,240401,24200004,24000000,PREGÃO ELETRÔNICO,Constitui objeto deste contrato o Serviço de S...,CONTRATO,20240404.1311137.Integra.CONTRATO.pdf,...,2024-03-20T00:00:00.000-03:00,NaN,False,LUCIANO PAMPLONA DE GOES CAVALCANTI,0001-01-01,6154,False,NaN,NaN,NaN
3,546438,240001,875801,240401,24200004,24000000,PREGÃO ELETRÔNICO,SOLICITAÇÃO PARA CONTRATUALIZAR O SALDO DISPON...,CONTRATO,20240321.1310031.Integra.CONTRATO.pdf,...,2024-03-27T00:00:00.000-03:00,NaN,False,CAMILLA DA SILVA MACIEL,0001-01-01,5354,False,NaN,NaN,NaN
4,546049,211101,922462,211101,56200006,56000000,PREGÃO ELETRÔNICO,COLHER DESCARTÁVEL PARA ADAGRI,DESPESA.SEM.INSTRUMENTO.CONTRATUAL,Sem Íntegra,...,2024-03-20T00:00:00.000-03:00,NaN,False,NaN,NaN,5007,False,NaN,NaN,NaN


##  Tratamento e Padronização dos Dados

In [67]:

# Função para normalizar nomes de colunas (minúsculas, sem acento, sem espaços)
def normalize_col(col):
    s = str(col)
    s = unicodedata.normalize('NFKD', s).encode('ASCII', 'ignore').decode('ASCII')
    s = s.lower().strip()
    s = re.sub(r'\s+', '_', s)
    s = re.sub(r'[^0-9a-z_]', '', s)
    return s

df_all.columns = [normalize_col(c) for c in df_all.columns]

# Mapeamento de colunas mais relevantes
mapping = {
    "orgao": ["orgao", "orgao_nome", "orgao_contratante"],
    "descricao_objeto": ["descricao_objeto", "objeto"],
    "descricao_modalidade": ["descricao_modalidade", "modalidade"],
    "data_assinatura": ["data_assinatura", "dt_assinatura"],
    "data_termino": ["data_termino", "dt_termino"],
    "valor_contrato": ["valor_contrato", "valor_total"],
    "valor_pago": ["valor_pago", "valor_liquidado"],
    "numero_contrato": ["numero_contrato", "num_contrato"]
}

# Aplica mapeamento
df_std = df_all.copy()
for std_col, candidates in mapping.items():
    df_std[std_col] = np.nan
    for c in candidates:
        if c in df_std.columns:
            df_std[std_col] = df_std[c]
            break

df_std.head()


,id,cod_concedente,cod_financiador,cod_gestora,cod_orgao,cod_secretaria,descricao_modalidade,descricao_objeto,descricao_tipo,descricao_url,...,gestor_contrato,data_finalizacao_prestacao_contas,sequential,emergency,law,has_non_profit_transfer,nome_fiscal,orgao,valor_pago,numero_contrato
0,546307,270001,891633,270401,27200004,27000000,NaN,NaN,CONTRATO,20240321.1310038.Integra.CONTRATO.pdf,...,CAMILA VIEIRA DA SILVA,NaN,5257,False,NaN,NaN,NaN,NaN,NaN,NaN
1,546632,228549,904377,220001,22000000,22000000,NaN,NaN,CONTRATO,20240321.1309998.Integra.CONTRATO.pdf,...,ROSANGELA PAIXAO VIEIRA DA SILVA,NaN,5531,False,NaN,NaN,NaN,NaN,NaN,NaN
2,547282,240301,914435,240401,24200004,24000000,NaN,NaN,CONTRATO,20240404.1311137.Integra.CONTRATO.pdf,...,LUCIANO PAMPLONA DE GOES CAVALCANTI,0001-01-01,6154,False,NaN,NaN,NaN,NaN,NaN,NaN
3,546438,240001,875801,240401,24200004,24000000,NaN,NaN,CONTRATO,20240321.1310031.Integra.CONTRATO.pdf,...,CAMILLA DA SILVA MACIEL,0001-01-01,5354,False,NaN,NaN,NaN,NaN,NaN,NaN
4,546049,211101,922462,211101,56200006,56000000,NaN,NaN,DESPESA.SEM.INSTRUMENTO.CONTRATUAL,Sem Íntegra,...,NaN,NaN,5007,False,NaN,NaN,NaN,NaN,NaN,NaN


## Conversão de Valores e Datas

In [69]:

# Conversão de moeda para float
def parse_currency(v):
    if pd.isna(v):
        return np.nan
    v = str(v).replace("R$", "").replace(".", "").replace(",", ".")
    v = re.sub(r'[^0-9.-]', '', v)
    try:
        return float(v)
    except:
        return np.nan

df_std["valor_contrato"] = df_std["valor_contrato"].apply(parse_currency)
df_std["valor_pago"] = df_std["valor_pago"].apply(parse_currency)

# Conversão de datas
df_std["data_assinatura"] = pd.to_datetime(df_std["data_assinatura"], errors="coerce", dayfirst=True)
df_std["data_termino"] = pd.to_datetime(df_std["data_termino"], errors="coerce", dayfirst=True)

# Colunas derivadas
df_std["duracao_dias"] = (df_std["data_termino"] - df_std["data_assinatura"]).dt.days
df_std["percentual_pago"] = np.where(df_std["valor_contrato"] > 0,
                                     (df_std["valor_pago"] / df_std["valor_contrato"]) * 100,
                                     np.nan)

df_std.head()


,id,cod_concedente,cod_financiador,cod_gestora,cod_orgao,cod_secretaria,descricao_modalidade,descricao_objeto,descricao_tipo,descricao_url,...,sequential,emergency,law,has_non_profit_transfer,nome_fiscal,orgao,valor_pago,numero_contrato,duracao_dias,percentual_pago
0,546307,270001,891633,270401,27200004,27000000,NaN,NaN,CONTRATO,20240321.1310038.Integra.CONTRATO.pdf,...,5257,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,546632,228549,904377,220001,22000000,22000000,NaN,NaN,CONTRATO,20240321.1309998.Integra.CONTRATO.pdf,...,5531,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,547282,240301,914435,240401,24200004,24000000,NaN,NaN,CONTRATO,20240404.1311137.Integra.CONTRATO.pdf,...,6154,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,546438,240001,875801,240401,24200004,24000000,NaN,NaN,CONTRATO,20240321.1310031.Integra.CONTRATO.pdf,...,5354,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,546049,211101,922462,211101,56200006,56000000,NaN,NaN,DESPESA.SEM.INSTRUMENTO.CONTRATUAL,Sem Íntegra,...,5007,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Análise Exploratória e Visualizações

In [70]:

# Top órgãos por valor contratado
orgao_col = "orgao"
valor_col = "valor_contrato"

# Se as colunas padronizadas estiverem vazias, retorne ao primeiro candidato disponível do mapeamento
if df_std[orgao_col].isna().all():
	for c in mapping[orgao_col]:
		if c in df_std.columns and not df_std[c].isna().all():
			orgao_col = c
			break

if df_std[valor_col].isna().all():
	for c in mapping[valor_col]:
		if c in df_std.columns and not df_std[c].isna().all():
			valor_col = c
			break

# Remover linhas com valores ausentes em qualquer coluna
df_plot = df_std[[orgao_col, valor_col]].dropna()

if not df_plot.empty:
	top_orgao = df_plot.groupby(orgao_col)[valor_col].apply(pd.to_numeric, errors='coerce').sum(level=orgao_col).sort_values(ascending=False).head(10)
	plt.figure(figsize=(10,6))
	top_orgao.plot(kind="barh")
	plt.title("Top 10 Órgãos por Valor Contratado")
	plt.xlabel("Valor Total (R$)")
	plt.show()
else:
	print("Nenhum dado disponível para plotar o gráfico de órgãos por valor contratado.")


Nenhum dado disponível para plotar o gráfico de órgãos por valor contratado.


In [72]:

# Distribuição de modalidades
modalidade_counts = df_std["descricao_modalidade"].value_counts().head(10)

if not modalidade_counts.empty:
	plt.figure(figsize=(10,6))
	modalidade_counts.plot(kind="bar")
	plt.title("Top 10 Modalidades")
	plt.ylabel("Quantidade")
	plt.show()
else:
	print("Nenhum dado disponível para plotar o gráfico de modalidades.")


Nenhum dado disponível para plotar o gráfico de modalidades.


## Exportação do Relatório

In [ ]:

output_csv = "relatorio_diario_contratos.csv"
df_std.to_csv(output_csv, index=False)
print("📂 Arquivo salvo:", output_csv)


## Gravação no Banco de Dados (SQL Server)

In [ ]:

odbc_conn_str = (
    'DRIVER={ODBC Driver 18 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=DMD;'
    'UID=sa;'
    'PWD=arte171721;'
    'Encrypt=yes;'
    'TrustServerCertificate=yes;'
)

if has_sqlalchemy:
    try:
        from sqlalchemy import create_engine
        engine_url = f"mssql+pyodbc:///?odbc_connect={quote_plus(odbc_conn_str)}"
        engine = create_engine(engine_url)
        df_std.to_sql("contratos_convenios", engine, if_exists="replace", index=False)
        print("✅ Dados gravados no SQL Server (tabela: contratos_convenios).")
    except Exception as e:
        print("❌ Erro ao gravar no SQL Server:", e)
else:
    print("⚠️ SQLAlchemy não disponível neste ambiente.")


## Envio Automático do Relatório por E-mail (CSV + Gráficos)

In [ ]:

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

EMAIL_REMETENTE = "artenio.reis@gmail.com"
SENHA_REMETENTE = "notj kuab ldzn gtzi"  # senha de app
EMAIL_DESTINATARIO = "artenioreis@live.com"

mensagem = MIMEMultipart()
mensagem['From'] = EMAIL_REMETENTE
mensagem['To'] = EMAIL_DESTINATARIO
mensagem['Subject'] = "📊 Relatório Diário de Contratos e Convênios"

corpo = """
<p>Olá,</p>
<p>Segue em anexo o relatório diário de contratos e convênios extraídos da plataforma Ceará Transparente.</p>
<p>Atenciosamente,<br>Artenio Reis</p>
"""
mensagem.attach(MIMEText(corpo, 'html'))

# Arquivos para enviar (CSV + gráficos)
arquivos = ["relatorio_diario_contratos.csv", "graf_top_orgao.png", "graf_modalidade.png"]

for arquivo in arquivos:
    if os.path.exists(arquivo):
        with open(arquivo, "rb") as anexo:
            parte = MIMEBase('application', 'octet-stream')
            parte.set_payload(anexo.read())
            encoders.encode_base64(parte)
            parte.add_header('Content-Disposition', f'attachment; filename={os.path.basename(arquivo)}')
            mensagem.attach(parte)

try:
    with smtplib.SMTP('smtp.gmail.com', 587) as servidor:
        servidor.starttls()
        servidor.login(EMAIL_REMETENTE, SENHA_REMETENTE)
        servidor.send_message(mensagem)
        print("✅ E-mail enviado com sucesso!")
except Exception as e:
    print("❌ Erro ao enviar e-mail:", e)



## ✅ Conclusão

Neste projeto foram aplicados os seguintes passos:
- União e padronização das bases de contratos e convênios
- Limpeza de dados (valores, datas e duplicatas)
- Criação de colunas derivadas para análise (ex.: duração, percentual pago)
- Análises gráficas (top órgãos, modalidades)
- Exportação de relatório consolidado em CSV
- Gravação dos dados no SQL Server
- Envio automático de relatório por e-mail (com CSV + gráficos anexados)

Este fluxo pode ser **agendado diariamente** (via Task Scheduler no Windows ou cron no Linux), garantindo transparência e acompanhamento contínuo.
